# Exploratory Data Analysis

First pass at analysing the output of blasting AML RNA-Seq data against the three tryptase sequences from Jonathon.


In [1]:
import pysam
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
def get_files(folder):
    
    return glob.glob('{folder}/*sorted.bam'.format(folder=folder))

In [3]:
list_of_control_results = get_files('../control_results/')

In [4]:
control_file_names = [x.split('/')[len(x.split('/'))-1] for x in list_of_control_results]

In [5]:
df_control = pd.DataFrame(index=control_file_names)

In [6]:
df_control['file_location'] = list_of_control_results

In [7]:
df_control['source'] = 'control'

In [8]:
df_control.head()

,file_location,source
aml_aabspliced_ERR1050073.sorted.bam,../control_results/aml_aabspliced_ERR1050073.s...,control
aml_aabspliced_ERR1050074.sorted.bam,../control_results/aml_aabspliced_ERR1050074.s...,control
aml_aabspliced_ERR1050075.sorted.bam,../control_results/aml_aabspliced_ERR1050075.s...,control
aml_aabspliced_ERR1050076.sorted.bam,../control_results/aml_aabspliced_ERR1050076.s...,control
aml_aabspliced_ERR1050077.sorted.bam,../control_results/aml_aabspliced_ERR1050077.s...,control


In [9]:
df = df_control

In [10]:
df.count()

file_location    1273
source           1273
dtype: int64

In [11]:
def get_read_count(df):

    return int(pysam.view("-c", df['file_location']))

In [12]:
df['alignment_count'] = df.apply(get_read_count, axis=1)

In [13]:
df.head()

,file_location,source,alignment_count
aml_aabspliced_ERR1050073.sorted.bam,../control_results/aml_aabspliced_ERR1050073.s...,control,2
aml_aabspliced_ERR1050074.sorted.bam,../control_results/aml_aabspliced_ERR1050074.s...,control,14
aml_aabspliced_ERR1050075.sorted.bam,../control_results/aml_aabspliced_ERR1050075.s...,control,3
aml_aabspliced_ERR1050076.sorted.bam,../control_results/aml_aabspliced_ERR1050076.s...,control,11
aml_aabspliced_ERR1050077.sorted.bam,../control_results/aml_aabspliced_ERR1050077.s...,control,5


In [14]:
def get_transcript_count(df, transcript_name):
    
    """
    Return the count of the transcript i.e how many of each of the three are in there.
    Note - Does not look at the quality of the alignment.
    
    """
    
    sam_file_location = df['file_location']
    
    samfile = pysam.AlignmentFile(sam_file_location, "rb")
    
    count = 0
    
    for read in samfile:
        
        if read is not None and read.reference_name == transcript_name:
            
            count = count + 1
            
    return count

In [15]:
df['alpha_wt_count'] = df.apply(get_transcript_count, axis=1, args=['Alpha_GEX_64k_HEX'])

In [16]:
df['alpha_dup_count'] = df.apply(get_transcript_count, axis=1, args=['Alpha_GEX_79k_dup_FAM'])
df['beta_count'] = df.apply(get_transcript_count, axis=1, args=['BETA_new_GEX_FAM'])

In [17]:
df.head()

,file_location,source,alignment_count,alpha_wt_count,alpha_dup_count,beta_count
aml_aabspliced_ERR1050073.sorted.bam,../control_results/aml_aabspliced_ERR1050073.s...,control,2,0,0,2
aml_aabspliced_ERR1050074.sorted.bam,../control_results/aml_aabspliced_ERR1050074.s...,control,14,4,4,6
aml_aabspliced_ERR1050075.sorted.bam,../control_results/aml_aabspliced_ERR1050075.s...,control,3,1,1,1
aml_aabspliced_ERR1050076.sorted.bam,../control_results/aml_aabspliced_ERR1050076.s...,control,11,4,4,3
aml_aabspliced_ERR1050077.sorted.bam,../control_results/aml_aabspliced_ERR1050077.s...,control,5,1,1,3


In [18]:
def get_zero_edit_distance_count(df, transcript_name):
    
    """
    For a goven transcript e.g. Alpha_GEX_64k_HEX get how many of the matches have an NM tag of zero
    i.e. the match was exact.
    
    """
    
    sam_file_location = df['file_location']
    
    samfile = pysam.AlignmentFile(sam_file_location, "rb")
    
    count = 0
    
    for read in samfile:
        
        if read is not None and read.reference_name == transcript_name:
            
            edit_distance = read.get_tag('NM')
            
            if edit_distance == 0:
                
                count = count + 1
            
    return count

In [19]:
df['alpha_wt_zero_edit_count'] = df.apply(get_zero_edit_distance_count, axis=1, args=['Alpha_GEX_64k_HEX'])
df['alpha_dup_zero_edit_count'] = df.apply(get_zero_edit_distance_count, axis=1, args=['Alpha_GEX_79k_dup_FAM'])
df['beta_zero_edit_count'] = df.apply(get_zero_edit_distance_count, axis=1, args=['BETA_new_GEX_FAM'])

In [20]:
df.head()

,file_location,source,alignment_count,alpha_wt_count,alpha_dup_count,beta_count,alpha_wt_zero_edit_count,alpha_dup_zero_edit_count,beta_zero_edit_count
aml_aabspliced_ERR1050073.sorted.bam,../control_results/aml_aabspliced_ERR1050073.s...,control,2,0,0,2,0,0,1
aml_aabspliced_ERR1050074.sorted.bam,../control_results/aml_aabspliced_ERR1050074.s...,control,14,4,4,6,4,4,6
aml_aabspliced_ERR1050075.sorted.bam,../control_results/aml_aabspliced_ERR1050075.s...,control,3,1,1,1,1,1,1
aml_aabspliced_ERR1050076.sorted.bam,../control_results/aml_aabspliced_ERR1050076.s...,control,11,4,4,3,4,4,3
aml_aabspliced_ERR1050077.sorted.bam,../control_results/aml_aabspliced_ERR1050077.s...,control,5,1,1,3,1,1,3


In [21]:
def get_transcript_read_count_filtered(df, transcript_name, start, end):
    
    """
    Count hits which cover the bit of the reference we are interested in
    
    """
    
    sam_file_location = df['file_location']
    
    samfile = pysam.AlignmentFile(sam_file_location, "rb")
    
    iter = samfile.fetch(transcript_name, start, end)
    
    count =0
    
    for read in iter:
        
        if read.reference_start <=start and read.reference_end >= end:
            
            count = count +1
            
    return count

In [22]:
df['alpha_read_covers_snps_count'] = df.apply(get_transcript_read_count_filtered,
                                              axis=1,
                                              args=['Alpha_GEX_64k_HEX', 25,45])

In [23]:
df['alpha_dup_read_covers_snps_count'] = df.apply(get_transcript_read_count_filtered,
                                              axis=1,
                                              args=['Alpha_GEX_79k_dup_FAM', 25,45])

df['beta_read_covers_snps_count'] = df.apply(get_transcript_read_count_filtered,
                                              axis=1,
                                              args=['BETA_new_GEX_FAM', 25,45])

In [24]:
df.sort_values(by='alpha_dup_read_covers_snps_count', ascending=False).head()

,file_location,source,alignment_count,alpha_wt_count,alpha_dup_count,beta_count,alpha_wt_zero_edit_count,alpha_dup_zero_edit_count,beta_zero_edit_count,alpha_read_covers_snps_count,alpha_dup_read_covers_snps_count,beta_read_covers_snps_count
aml_aabspliced_SRR2094338.sorted.bam,../control_results/aml_aabspliced_SRR2094338.s...,control,87,30,29,28,20,20,28,2,1,0
aml_aabspliced_ERR1050073.sorted.bam,../control_results/aml_aabspliced_ERR1050073.s...,control,2,0,0,2,0,0,1,0,0,0
aml_aabspliced_ERR188435.sorted.bam,../control_results/aml_aabspliced_ERR188435.so...,control,1,0,0,1,0,0,1,0,0,0
aml_aabspliced_ERR188413.sorted.bam,../control_results/aml_aabspliced_ERR188413.so...,control,5,2,2,1,2,2,1,0,0,0
aml_aabspliced_ERR188412.sorted.bam,../control_results/aml_aabspliced_ERR188412.so...,control,1,0,0,1,0,0,0,0,0,0


In [25]:
def get_transcript_read_count_filtered_exact(df, transcript_name, start, end):
    
    """
    Count hits which cover the bit of the reference we are interested in and which are exact.
    
    That is do they cross position 0 - 44 of the transcript ( given by transcipt_name) and \
    have an edit distance from the reference of 0. 
    
    Reads which cross this location will cross all four SNPs needed to tell the 3 transcripts apart.
    
    """
    
    sam_file_location = df['file_location']
    
    samfile = pysam.AlignmentFile(sam_file_location, "rb")
    
    iter = samfile.fetch(transcript_name, start, end)
    
    count =0
    
    for read in iter:
        
        if read.reference_start <=start and read.reference_end >=end:
            
            edit_distance = read.get_tag('NM')
            
            if edit_distance == 0:
                
                count = count + 1
                
    return count

In [26]:
df['alpha_read_covers_snps_count_exact'] = df.apply(get_transcript_read_count_filtered_exact,
                                              axis=1,
                                              args=['Alpha_GEX_64k_HEX', 25,45])

df['alpha_dup_read_covers_snps_count_exact'] = df.apply(get_transcript_read_count_filtered_exact,
                                              axis=1,
                                              args=['Alpha_GEX_79k_dup_FAM', 25,45])

df['beta_read_covers_snps_count_exact'] = df.apply(get_transcript_read_count_filtered_exact,
                                              axis=1,
                                              args=['BETA_new_GEX_FAM', 25,45])

In [27]:
# Quick check to see if any such reads exist

#df.sort_values(by='alpha_dup_read_covers_snps_count_exact', ascending=False).head()

df.head()

,file_location,source,alignment_count,alpha_wt_count,alpha_dup_count,beta_count,alpha_wt_zero_edit_count,alpha_dup_zero_edit_count,beta_zero_edit_count,alpha_read_covers_snps_count,alpha_dup_read_covers_snps_count,beta_read_covers_snps_count,alpha_read_covers_snps_count_exact,alpha_dup_read_covers_snps_count_exact,beta_read_covers_snps_count_exact
aml_aabspliced_ERR1050073.sorted.bam,../control_results/aml_aabspliced_ERR1050073.s...,control,2,0,0,2,0,0,1,0,0,0,0,0,0
aml_aabspliced_ERR1050074.sorted.bam,../control_results/aml_aabspliced_ERR1050074.s...,control,14,4,4,6,4,4,6,0,0,0,0,0,0
aml_aabspliced_ERR1050075.sorted.bam,../control_results/aml_aabspliced_ERR1050075.s...,control,3,1,1,1,1,1,1,0,0,0,0,0,0
aml_aabspliced_ERR1050076.sorted.bam,../control_results/aml_aabspliced_ERR1050076.s...,control,11,4,4,3,4,4,3,0,0,0,0,0,0
aml_aabspliced_ERR1050077.sorted.bam,../control_results/aml_aabspliced_ERR1050077.s...,control,5,1,1,3,1,1,3,0,0,0,0,0,0


## Sanity Check

Print out some of the reads. I then did a manual BLAST and alignment to check the code was working as expected

In [28]:
def get_transcript_read_count_filtered_exact_print(sam_file_location, transcript_name, start, end):
    
    """
    Same as get_transcript_read_count_filtered_exact() except we just print out the reads instead of \
    counting them.
    
    
    """
    
    samfile = pysam.AlignmentFile(sam_file_location, "rb")
    
    iter = samfile.fetch(transcript_name, start, end)
    
    for read in iter:
        
        if read.reference_start <=start and read.reference_end >= end:
            
            edit_distance = read.get_tag('NM')
            
            if edit_distance == 0:
                
                print (read)

In [29]:
#Pick a random bam file with some read alignments in this area

get_transcript_read_count_filtered_exact_print('../aml_results/aml_aabspliced_SRR5626188.sam.sorted.bam',
                                               'Alpha_GEX_64k_HEX',
                                               25,
                                               45 )

SRR5626188.47984736	163	0	0	255	7S94M	0	5	94	GGGAGAGCCCGCTGGGTAGAAGGAACAGGGAGTGGCCAGGATGCTGAGCCTGCTGCTGCTGGCGCTGCCCGTCCTGGCGAGCCGCGCCTACGCGGCCCCTG	None	[('NH', 1), ('AS', 94), ('NM', 0)]
SRR5626188.59402515	163	0	0	255	6S95M	0	150	95	GGAGAGCCCGCTGGGTAGAAGGAACAGGGAGTGGCCAGGATGCTGAGCCTGCTGCTGCTGGCGCTGCCCGTCCTGGCGAGCCGCGCCTACGCGGCCCCTGC	None	[('NH', 1), ('AS', 95), ('NM', 0)]
SRR5626188.63045359	163	0	0	255	24S77M	0	18	77	GAAGGATAAATGGGGAGGGGAGAGCCCGCTGGGTAGAAGGAACAGGGAGTGGCCAGGATGCTGAGCCTGCTGCTGCTGGCGCTGCCCGTCCTGGCGAGCCG	None	[('NH', 1), ('AS', 77), ('NM', 0)]
SRR5626188.63657800	163	0	0	255	6S95M	0	125	95	GGAGAGCCCGCTGGGTAGAAGGAACAGGGAGTGGCCAGGATGCTGAGCCTGCTGCTGCTGGCGCTGCCCGTCCTGGCGAGCCGCGCCTACGCGGCCCCTGC	None	[('NH', 1), ('AS', 95), ('NM', 0)]
SRR5626188.68846470	99	0	0	255	6S95M	0	41	95	GGAGAGCCCGCTGGGTAGAAGGAACAGGGAGTGGCCAGGATGCTGAGCCTGCTGCTGCTGGCGCTGCCCGTCCTGGCGAGCCGCGCCTACGCGGCCCCTGC	None	[('NH', 1), ('AS', 95), ('NM', 0)]
SRR5626188.16757617	89	0	0	255	41S60M	-1	-1	60	CGCCCCCTCCTG

In [30]:
df.describe()

,alignment_count,alpha_wt_count,alpha_dup_count,beta_count,alpha_wt_zero_edit_count,alpha_dup_zero_edit_count,beta_zero_edit_count,alpha_read_covers_snps_count,alpha_dup_read_covers_snps_count,beta_read_covers_snps_count,alpha_read_covers_snps_count_exact,alpha_dup_read_covers_snps_count_exact,beta_read_covers_snps_count_exact
count,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.000000,1273.0,1273.000000
mean,10.109976,3.273370,3.241163,3.595444,2.560880,2.543598,3.243519,0.016496,0.000786,0.004713,0.003928,0.0,0.004713
std,34.134522,11.244805,11.144173,12.005458,8.329946,8.304155,11.223099,0.150088,0.028028,0.068518,0.062573,0.0,0.068518
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,334.000000,106.000000,105.000000,123.000000,93.000000,93.000000,115.000000,2.000000,1.000000,1.000000,1.000000,0.0,1.000000


In [32]:
df.to_csv('data.control.3snps.csv')

In [33]:
df['alignment_count'].sum()

12870